In [ ]:
!pip install streamlit faker


In [ ]:
#create synthetic data
import random
from faker import Faker

fake = Faker()

def generate_synthetic_logs(num_logs):
    users = ['rbinnajim.c', 'nmzuabi', 'mhassanien.c', 'sadmin']
    logs = []

    for _ in range(num_logs):
        user = random.choice(users)
        ip = fake.ipv4()
        session_id = fake.uuid4()
        timestamp = fake.date_time_this_month()

        # Log for LDAP Authentication
        log = f"{timestamp} LDAP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        # OTP bypass for specific users like 'sadmin', otherwise OTP success
        if user != 'sadmin':
            log += f"{timestamp} OTP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        log += f"{timestamp} Session ID[{session_id}], Authentication Done. Redirecting to Siebel\n"
        logs.append(log)

    return '\n'.join(logs)


In [ ]:
import streamlit as st
import re

# Step 1: Generate synthetic logs and allow download
@st.cache_data
def generate_synthetic_logs(num_logs):
    from faker import Faker
    fake = Faker()

    users = ['rbinnajim.c', 'nmzuabi', 'mhassanien.c', 'sadmin']
    logs = []

    for _ in range(num_logs):
        user = random.choice(users)
        ip = fake.ipv4()
        session_id = fake.uuid4()
        timestamp = fake.date_time_this_month()

        # Log for LDAP Authentication
        log = f"{timestamp} LDAP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        # OTP bypass for specific users like 'sadmin', otherwise OTP success
        if user != 'sadmin':
            log += f"{timestamp} OTP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        log += f"{timestamp} Session ID[{session_id}], Authentication Done. Redirecting to Siebel\n"
        logs.append(log)

    return '\n'.join(logs)

# Step 2: Parse uploaded log data
def parse_logs(log_data):
    # Define patterns to match LDAP, OTP successes, and bypass attempts
    pattern = r"LDAP Authentication SUCCESS for User\[(.*?)\]\. Client IP\[(.*?)\]"
    ldap_matches = re.findall(pattern, log_data)

    otp_pattern = r"OTP Authentication SUCCESS for User\[(.*?)\]\. Client IP\[(.*?)\]"
    otp_matches = re.findall(otp_pattern, log_data)

    bypass_pattern = r"LDAP Authentication SUCCESS for User\[(.*?)\] .*? Redirecting to Siebel"
    bypass_matches = re.findall(bypass_pattern, log_data)

    return ldap_matches, otp_matches, bypass_matches

# Step 3: Streamlit app logic
def main():
    st.title("Login Log Analysis Tool")

    # Generate synthetic logs and allow user to download them
    st.sidebar.header("Generate Synthetic Logs")
    num_logs = st.sidebar.slider("Number of Logs", 1, 100, 10)

    if st.sidebar.button("Generate Logs"):
        synthetic_logs = generate_synthetic_logs(num_logs)
        st.sidebar.download_button(
            label="Download Synthetic Logs",
            data=synthetic_logs,
            file_name="synthetic_logs.txt"
        )

    # Upload log file section
    st.header("Upload Log File")
    uploaded_file = st.file_uploader("Choose a log file", type=["txt"])

    if uploaded_file is not None:
        # Read the log data
        log_data = uploaded_file.read().decode("utf-8")

        # Display raw logs
        st.subheader("Uploaded Log Data")
        st.text_area("Logs", log_data, height=300)

        # Parse the logs
        ldap_matches, otp_matches, bypass_matches = parse_logs(log_data)

        # Display findings
        st.subheader("Analysis of Logs")

        st.write("### LDAP Authentication Successes")
        if ldap_matches:
            for user, ip in ldap_matches:
                st.write(f"User: {user}, IP: {ip}")
        else:
            st.write("No LDAP Authentication Successes found.")

        st.write("### OTP Authentication Successes")
        if otp_matches:
            for user, ip in otp_matches:
                st.write(f"User: {user}, IP: {ip}")
        else:
            st.write("No OTP Authentication Successes found.")

        st.write("### Bypass Attempts (No OTP)")
        if bypass_matches:
            for user, ip in bypass_matches:
                st.write(f"User: {user}, IP: {ip} (OTP Bypass)")
        else:
            st.write("No OTP Bypass Attempts found.")

# Run the app
if __name__ == "__main__":
    main()


In [ ]:
!pip install streamlit faker pandas numpy matplotlib plotly scikit-learn


In [ ]:
!pip install streamlit pandas matplotlib seaborn numpy python-docx openai==0.28

In [ ]:
#Working app code as on 5th Nov 2024
import random
from faker import Faker
import streamlit as st
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Step 1: Generate synthetic logs
def generate_synthetic_logs(num_logs):
    fake = Faker()
    users = ['rbinnajim.c', 'nmzuabi', 'mhassanien.c', 'sadmin']
    logs = []
    now = datetime.now()

    for _ in range(num_logs):
        timestamp = now - timedelta(minutes=random.randint(1, 10000))
        user = random.choice(users)
        ip = fake.ipv4()
        session_id = fake.uuid4()

        # Log for LDAP Authentication
        log = f"{timestamp} LDAP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        if user != 'sadmin':
            log += f"{timestamp} OTP Authentication SUCCESS for User[{user}]. Client IP[{ip}]\n"

        log += f"{timestamp} Session ID[{session_id}], Authentication Done. Redirecting to Siebel\n"
        logs.append(log)

    return '\n'.join(logs)

# Step 2: Parse logs
def parse_logs(log_data):
    ldap_pattern = r"(\d+-\d+-\d+ \d+:\d+:\d+).*?LDAP Authentication SUCCESS for User\[(.*?)\]"
    otp_pattern = r"(\d+-\d+-\d+ \d+:\d+:\d+).*?OTP Authentication SUCCESS for User\[(.*?)\]"

    ldap_matches = re.findall(ldap_pattern, log_data)
    otp_matches = re.findall(otp_pattern, log_data)

    ldap_df = pd.DataFrame(ldap_matches, columns=['timestamp', 'user'])
    ldap_df['timestamp'] = pd.to_datetime(ldap_df['timestamp'])
    otp_df = pd.DataFrame(otp_matches, columns=['timestamp', 'user'])
    otp_df['timestamp'] = pd.to_datetime(otp_df['timestamp'])

    return ldap_df, otp_df

# Step 3: Visualize log data
def visualize_data(ldap_df, otp_df):
    st.subheader("Login Trends Over Time")
    st.write("This graph shows the trend of LDAP authentication success events over time, helping to observe if login activities have increased or decreased.")

    # Plot LDAP Authentication Trend
    ldap_df['date'] = ldap_df['timestamp'].dt.date
    ldap_counts = ldap_df.groupby('date').size()

    fig = px.line(ldap_counts, title="LDAP Authentication Success Trend")
    st.plotly_chart(fig)

    # Plot OTP Authentication Trend
    otp_df['date'] = otp_df['timestamp'].dt.date
    otp_counts = otp_df.groupby('date').size()

    st.write("This graph shows the trend of OTP authentication success events, which helps in analyzing the additional security verification activities.")
    fig = px.line(otp_counts, title="OTP Authentication Success Trend", color_discrete_sequence=['green'])
    st.plotly_chart(fig)

    # Identify and plot anomalies
    st.subheader("Anomalies Detection")
    st.write("The bar chart below shows the hourly counts of LDAP authentication events. Bars in red highlight any anomalies where login activity significantly exceeds the norm.")

    ldap_df['hour'] = ldap_df['timestamp'].dt.hour
    hourly_counts = ldap_df.groupby('hour').size()

    threshold = hourly_counts.mean() + 2 * hourly_counts.std()  # basic anomaly threshold
    anomalies = hourly_counts[hourly_counts > threshold]

    fig, ax = plt.subplots()
    hourly_counts.plot(ax=ax, color="blue", label="Hourly Counts")
    anomalies.plot(ax=ax, kind="bar", color="red", label="Anomalies")
    plt.axhline(y=threshold, color='orange', linestyle='--', label='Threshold')
    plt.legend()
    st.pyplot(fig)

    # Prediction of LDAP authentication trend
    st.subheader("Trend Prediction")
    st.write("This graph provides a forecast for LDAP authentication events based on past data trends, giving insight into expected future login activities.")

    ldap_counts.index = pd.to_datetime(ldap_counts.index)
    ldap_counts = ldap_counts.reset_index()
    ldap_counts['date_ordinal'] = ldap_counts['date'].apply(lambda x: x.toordinal())

    X = np.array(ldap_counts['date_ordinal']).reshape(-1, 1)
    y = np.array(ldap_counts[0])

    model = LinearRegression()
    model.fit(X, y)
    ldap_counts['predicted'] = model.predict(X)

    fig = px.line(ldap_counts, x='date', y='predicted', title="Predicted LDAP Authentication Trend")
    fig.add_scatter(x=ldap_counts['date'], y=ldap_counts[0], mode='lines', name='Actual')
    st.plotly_chart(fig)

    mse = mean_squared_error(y, ldap_counts['predicted'])
    st.write(f"Prediction Mean Squared Error: {mse:.2f}")

# Step 4: Q&A Interface
def handle_question(question, ldap_df, otp_df):
    question = question.lower()

    if "trend in logins" in question:
        response = "The trend shows the daily count of successful LDAP and OTP logins. Generally, an increase indicates more usage or activity, while a decrease could suggest fewer access attempts."
    elif "anomalies" in question:
        response = "Anomalies are detected where hourly login counts exceed a threshold based on average activity. These may indicate unusual access attempts or periods of high demand."
    elif "prediction" in question:
        response = "The prediction graph forecasts future LDAP login events using a linear model, helping to anticipate future login patterns."
    else:
        response = "Sorry, I didn't understand the question. Try asking about 'trend in logins,' 'anomalies,' or 'prediction'."

    return response

# Step 5: Streamlit App
def main():
    st.title("Enhanced Login Log Analysis Tool with Q&A Interface")

    # Generate synthetic logs
    st.sidebar.header("Generate Synthetic Logs")
    num_logs = st.sidebar.slider("Number of Logs", 1, 1000, 100)

    if st.sidebar.button("Generate Logs"):
        synthetic_logs = generate_synthetic_logs(num_logs)
        st.sidebar.download_button(
            label="Download Synthetic Logs",
            data=synthetic_logs,
            file_name="synthetic_logs.txt"
        )

    # Upload log file section
    st.header("Upload Log File")
    uploaded_file = st.file_uploader("Choose a log file", type=["txt"])

    if uploaded_file is not None:
        log_data = uploaded_file.read().decode("utf-8")
        ldap_df, otp_df = parse_logs(log_data)

        st.subheader("Raw Log Data")
        st.text_area("Logs", log_data, height=200)

        # Display parsed data tables
        st.subheader("Parsed LDAP Log Data")
        st.dataframe(ldap_df)

        st.subheader("Parsed OTP Log Data")
        st.dataframe(otp_df)

        # Visualize data with trends, anomalies, and predictions
        visualize_data(ldap_df, otp_df)

        # Q&A Interface
        st.subheader("Ask Questions About the Log Data")
        user_question = st.text_input("Enter your question about the log data:")

        if user_question:
            response = handle_question(user_question, ldap_df, otp_df)
            st.write(response)

# Run the app
if __name__ == "__main__":
    main()


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 6s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [ ]:
!streamlit run app.py &> /content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com


35.221.196.118
⠙your url is: https://bright-frogs-shine.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:10103 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:517:28)
    at emitErrorNT (node:internal/streams/destroy:151:8)
    at emitErrorCloseNT (node:internal/streams/destroy:116:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v18.20.5
⠙